In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
model_name = "NousResearch/llama-2-7b-chat-hf" # use this if you have access to the official LLaMA 2 model "meta-llama/Llama-2-7b-chat-hf", though keep in mind you'll need to pass a Hugging Face key argument
dataset_name = "/content/train.jsonl"
new_model = "llama-2-7b-custom"
lora_r = 32
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 5
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

In [ ]:
system_prompt = (
    "You are an expert in solving math problems. You will be given two numbers "
    "and you need to add them. Do not produce any output other than the sum of the two numbers. "
    "Provide the answer as a single number without any text, explanations, or punctuation."
)

In [ ]:
# Load datasets
train_dataset = load_dataset('json', data_files=f'/content/{dataset_name}_train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files=f'/content/{dataset_name}_test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_prompt.strip()}\n<</SYS>>\n\n' + str(num1) + " + " + str(num2) + ' [/INST] ' + str(result) for num1,num2,result in zip(examples['num1'], examples['num2'], examples['result'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_prompt.strip()}\n<</SYS>>\n\n' + str(num1) + " + " + str(num2) + ' [/INST] ' + str(result) for num1,num2,result in zip(examples['num1'], examples['num2'], examples['result'])]}, batched=True)

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5  # Evaluate every 20 steps
)

In [ ]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

In [ ]:
trainer.train()

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
import pandas as pd
test_dataset = "k_digit_addition_test_dataset.csv"
val_dataset = "k_digit_addition_val_dataset.csv"

test_df = pd.read_csv(test_dataset)
val_df = pd.read_csv(val_dataset)

In [ ]:
import re

def extract_last_number(output):
    """
    Extracts the last number from the given string.
    
    Parameters:
        output (str): The string containing the output.
    
    Returns:
        int or None: The last number if found, otherwise None.
    """
    matches = re.findall(r'\d+', output)
    if matches:
        return int(matches[-1])  # Get the last match and convert to int
    return None

In [ ]:
from tqdm import tqdm

In [ ]:
num_eval = 10

results = []
predictions = []  # List to store detailed predictions

for k in tqdm(range(1, 13)):
    test_df_subset = test_df[test_df["digits"] == k].head(num_eval)
    val_df_subset = val_df[val_df["digits"] == k]

    few_shot_prompt = system_prompt + "\n\nExamples:\n"
    for _, row in val_df_subset.iterrows():
        few_shot_prompt += f"{row['num1']} + {row['num2']} = {row['result']}\n"

    correct = 0
    total = len(test_df_subset)

    for _, row in tqdm(test_df_subset.iterrows(), total=len(test_df_subset)):
        test_prompt = f"\n {row['num1']} + {row['num2']} ="

        prompt = f"""[INST] <<SYS>>\n{few_shot_prompt}\n<</SYS>>\n\n {test_prompt} Answer:[/INST]"""

        pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)

        result = pipe(prompt)

        text = result[0]['generated_text']

        model_output = text.split("[/INST]")[1].strip()

        extracted_num = extract_last_number(model_output)

        # Check if the model output matches the expected sum
        is_correct = extracted_num == int(row['result'])
        if is_correct:
            correct += 1
        
        # Store the prediction details
        predictions.append({
            "k": k,
            "num1": row['num1'],
            "num2": row['num2'],
            "correct_output": int(row['result']),
            "predicted_output": extracted_num,
            "is_correct": is_correct
        })

    # Record accuracy for this k
    accuracy = correct / total if total > 0 else 0
    print(k, accuracy)
    results.append({"k": k, "accuracy": accuracy})


In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv("k_digit_addition_results_finetune.csv", index=False)

predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv("k_digit_addition_predictions_finetune.csv", index=False)
